<a href="https://colab.research.google.com/github/seled/CI-ApplDataSci/blob/master/Coursera_Zurich_District_Restaurants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction/Business Problem

Zurich City, Switzerland becomes more and more a place for food with new restaurants opening every month. On the other hand, investment and operating costs for restaurants are very high. To be succesful, it is important to choose the right location. This report gives an overview of current restaurant offering by cuisine in the city's districts. Potential restaurant owner may use this information when considering locations to open new restaurants.

# Data
We will use data about restaurants from Foursquare and evaluate them according to their location (district) and cuisine. Additionaly we may use other data such as rent-prices for restaurant properties.

# Methodology
We First look into the separation of districts and for that we choose the official statistical districts. We can get data incl. geojson from the open data portal of the city of Zurich. In the same portal we also find data about officialy registered rstaurants per district. 

In a next step we include data from foursquare to have additional data of restaurant types. 

Finally we use KNN to cluster the different districts in the same way we did for New York, but with only Data about Restaurants.



# Results
The first discovery we made is that when looking at the distribution of restaurants, we have a high density in the "central" districts whereas a very low density in most of the "outer" districts. 

In a seconf step we use data from Foursquare to add more information about the restaurants. For this report, we focus on restaurant type, but we could also add other features. Unfortunately the amount of data from foursquare for this city/country is not that high. 

We can cluster the district with KNN according to the restaurant types to see where we have similar types of restaurants. 

# Discussion
Foursquare data is probably not the best source for this area/country. In a follow up, we should try to use data from google maps which seems to have more data and thus would lead to more features and better significance. 

# Conclusion
We can see that there are some centers with a lot of restaurants and variotions of restaurant types. Then we have some districts with very few restaurants where it's not stastistically significant. We need to identify further data such as renting prices etc. to give a better guidance on our initital question.

In terms of clustering, we have identified similar districts according to similar restaurant types and could use this to e.g. build a recommender for which restaurant type that is still missing could be popular in which district.

# Code

In [0]:
# Import Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import folium
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

In [0]:
# Zurich Geo Data
# https://data.stadt-zuerich.ch/dataset/d49f5604-6a27-460e-9568-db5d7ce5d4a6/resource/35e28fa3-d53d-4ec0-b7cd-8e3ac95f319a/download/stadtquartier.json
!wget --quiet https://data.stadt-zuerich.ch/dataset/d49f5604-6a27-460e-9568-db5d7ce5d4a6/resource/35e28fa3-d53d-4ec0-b7cd-8e3ac95f319a/download/stadtquartier.json -O stadtquartiere.json
districts_json = r'stadtquartiere.json'

In [0]:
!wget --quiet https://data.stadt-zuerich.ch/dataset/statistisches_quartier/resource/c837926e-035d-48b9-8656-03f1b13c323b/download/statistische_quartiere.json
print('GeoJSON file downloaded!')
statdistricts_json = r'statistische_quartiere.json'

GeoJSON file downloaded!


In [0]:
!wget --quiet https://data.stadt-zuerich.ch/dataset/b2c829d4-9d4b-4741-a944-242c28f00b2a/resource/e54ff412-23e3-4a85-bd0a-c07501ba02f0/download/gastwirtschaftsbetriebe_per_20181231.json
print('GeoJSON file downloaded!')
gastw_json = r'gastwirtschaftsbetriebe_per_20181231.json'

GeoJSON file downloaded!


In [0]:
#Show a map of Zurich and districts
latitude = 47.3769
longitude = 8.5417

zurich_map = folium.Map(location=[latitude, longitude], zoom_start=13)
folium.GeoJson(
    statdistricts_json,
    name='geojson'
).add_to(zurich_map)

zurich_map

**Restaurant Data**

In [0]:
# Registered Restaurants in Zurich Data from City of Zurich
URL = 'https://data.stadt-zuerich.ch/dataset/b2c829d4-9d4b-4741-a944-242c28f00b2a/resource/56cad69a-0ebb-407f-a83b-db8ea7e21bf8/download/gastwirtschaftsbetriebe_per_20181231.csv'
df_rest_zh = pd.read_csv(URL)


In [0]:
df_rest_zh

,Jahr,Betriebsname,StrasseLang,HNr,PLZ,Ort,StatZoneSort,StatZoneLang,QuarSort,QuarLang,KreisSort,KreisLang,Oeffnungszeit,Betriebsart,Betriebsstatus,EKoord,NKoord,AnzBetriebe
0,2018,01 Bar,Köngengasse,2,8001,Zürich,1102,Prediger,11,Rathaus,1,Kreis 1,0200/frei,Gastwirtschaft,Offen,2683418.307,1247572.417,1
1,2018,01 Ocakbasi,Bernerstrasse Nord,224,8064,Zürich,9207,Werdhölzli,92,Altstetten,9,Kreis 9,0400/Frei,Gastwirtschaft,Offen,2678790.445,1250030.915,1
2,2018,100%,Badenerstrasse,217,8003,Zürich,3405,Idaplatz,34,Sihlfeld,3,Kreis 3,NaN,Gastwirtschaft,Offen,2681506.383,1247709.748,1
3,2018,1001 GmbH,Niederdorfstrasse,4,8001,Zürich,1102,Prediger,11,Rathaus,1,Kreis 1,0400/frei,Gastwirtschaft,Offen,2683473.043,1247477.541,1
4,2018,169West,Weststrasse,169,8003,Zürich,3405,Idaplatz,34,Sihlfeld,3,Kreis 3,NaN,Gastwirtschaft,Offen,2681517.138,1247567.050,1
5,2018,2. Akt,Selnaustrasse,2,8001,Zürich,1403,Selnaustrasse,14,City,1,Kreis 1,täglich bis 0400 Uhr,Gastwirtschaft,Offen,2682700.759,1247135.169,1
6,2018,20/20 by Mövenpick Wein Restaurant,Nüschelerstrasse,1,8001,Zürich,1404,Paradeplatz,14,City,1,Kreis 1,NaN,Gastwirtschaft,Offen,2682989.611,1247273.348,1
7,2018,25hours Hotel Zürich Langstrasse,Lagerstrasse,110,8004,Zürich,4206,Kaserne,42,Langstrasse,4,Kreis 4,"So - Mi 01.00, Do - Sa 02.00",Gastwirtschaft,Offen,2682320.280,1248295.364,1
8,2018,25hours Hotel Zürich West,Pfingstweidstrasse,102,8005,Zürich,5201,Hardturm,52,Escher Wyss,5,Kreis 5,0400/Frei,Gastwirtschaft,Offen,2680873.320,1249459.137,1
9,2018,26 Rose Garden,Breitensteinstrasse,14,8037,Zürich,10202,Im Sydefädeli,102,Wipkingen,10,Kreis 10,NaN,Nebenwirtschaft,Offen,2681795.747,1249715.650,1


In [0]:
df_rest_quar = df_rest_zh.groupby('QuarLang')[['AnzBetriebe']].sum()
df_rest_quar = df_rest_quar.reset_index()
df_rest_quar


,QuarLang,AnzBetriebe
0,Affoltern,32
1,Albisrieden,33
2,Alt-Wiedikon,76
3,Altstetten,105
4,City,158
5,Enge,77
6,Escher Wyss,104
7,Fluntern,21
8,Friesenberg,22
9,Gewerbeschule,120


In [0]:
#### TO BE DELETED
import json

with open(statdistricts_json) as f:
    data = json.load(f)

x = []
for feature in data['features']:
    x.append(feature['properties']['qname'])
qname = pd.DataFrame(x)
len(x)

34

In [0]:
zurich_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.Choropleth(
    geo_data=statdistricts_json,
    data=df_rest_quar,
    columns=['QuarLang','AnzBetriebe'],
    key_on='properties.qname',
    fill_color='Purples',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of Restaurants'
).add_to(zurich_map)

zurich_map

**Restaurant Type - Foursquare**

In [0]:
CLIENT_ID = 'U3FNAHG45SZ3C2CDXQY1YCZEEO4HFGOSI1USTRY4PCL2FKTF' # your Foursquare ID
CLIENT_SECRET = 'IAOANM3KSR1CFXHUVAG23J4JWV1TAYJRGZOS0CJ1DHTIBKBF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: U3FNAHG45SZ3C2CDXQY1YCZEEO4HFGOSI1USTRY4PCL2FKTF
CLIENT_SECRET:IAOANM3KSR1CFXHUVAG23J4JWV1TAYJRGZOS0CJ1DHTIBKBF


In [0]:
quartier = "Altstetten"
categoryId='4d4b7105d754a06374d81259' # Food Category

In [0]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&categoryId={}&near={}&intent={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    categoryId,
    quartier,
    'browse',
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/search?&client_id=U3FNAHG45SZ3C2CDXQY1YCZEEO4HFGOSI1USTRY4PCL2FKTF&client_secret=IAOANM3KSR1CFXHUVAG23J4JWV1TAYJRGZOS0CJ1DHTIBKBF&v=20180605&categoryId=4d4b7105d754a06374d81259&near=Altstetten&intent=browse&radius=500&limit=100'

In [0]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cd32995db04f559db0daf94'},
 'response': {'geocode': {'feature': {'cc': 'CH',
    'displayName': 'Zürich (Kreis 9) / Altstetten, Switzerland',
    'geometry': {'center': {'lat': 47.38946, 'lng': 8.48533}},
    'highlightedName': '<b>Altstetten</b>, Switzerland',
    'id': 'geonameid:6295513',
    'longId': '72057594044223449',
    'matchedName': 'Altstetten, Switzerland',
    'name': 'Zürich (Kreis 9) / Altstetten',
    'slug': 'zuerich-kreis-9-/-altstetten-switzerland',
    'woeType': 22},
   'parents': [],
   'what': '',
   'where': 'altstetten'},
  'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gas_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d113951735',
      'name': 'Gas Station',
      'pluralName': 'Gas Stations',
      'primary': True,
      'shortName': 'Gas Station'}],
    'hasPerk': False,
    'id': '4c724942d944224b3f3a0c29',
    'location': {'address': 'Hohlstrasse 544',
     'cc

In [0]:
df_rest_quar

,QuarLang,AnzBetriebe
0,Affoltern,32
1,Albisrieden,33
2,Alt-Wiedikon,76
3,Altstetten,105
4,City,158
5,Enge,77
6,Escher Wyss,104
7,Fluntern,21
8,Friesenberg,22
9,Gewerbeschule,120


In [0]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [0]:
center = results['response']['geocode']['feature']['geometry']['center']
center

{'lat': 47.38946, 'lng': 8.48533}

[{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gas_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d113951735',
    'name': 'Gas Station',
    'pluralName': 'Gas Stations',
    'primary': True,
    'shortName': 'Gas Station'}],
  'hasPerk': False,
  'id': '4c724942d944224b3f3a0c29',
  'location': {'address': 'Hohlstrasse 544',
   'cc': 'CH',
   'city': 'Zürich',
   'country': 'Schweiz',
   'formattedAddress': ['Hohlstrasse 544', '8048 Zürich', 'Schweiz'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 47.39032762830163,
     'lng': 8.491520897359738}],
   'lat': 47.39032762830163,
   'lng': 8.491520897359738,
   'postalCode': '8048',
   'state': 'Zürich'},
  'name': 'Socar',
  'referralId': 'v-1557342613'},
 {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/burger_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d16c941735',
    'name': 'Burger Joint',
    'pluralName': 'Burger Joints',
    'primary': Tru

In [0]:
venues = results['response']['venues']

nearby_venues = json_normalize(venues) # flatten JSON
venues

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Socar,Gas Station,47.390328,8.491521
1,Burger Brothers,Burger Joint,47.387615,8.486487
2,La Taquería,Mexican Restaurant,47.388570,8.486356
3,Migros Restaurant,Restaurant,47.388440,8.486998
4,Scent of Bamboo,Chinese Restaurant,47.391232,8.489412
5,Caffè Spettacolo,Café,47.391236,8.488381
6,Dunkin' Donuts,Donut Shop,47.388386,8.487498
7,Memo Bar,Fast Food Restaurant,47.387783,8.486138
8,Santa Lucia,Italian Restaurant,47.388519,8.486808
9,SUBWAY,Sandwich Place,47.391057,8.489908


In [0]:
def getNearbyVenues(names, radius=1000, categoryId='4d4b7105d754a06374d81259'):
    
    venues_list=[]
    for name in names:
        print(name)
        # create the API request URL
        LIMIT=100
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&categoryId={}&near={}&intent={}&radius={}&limit={}'.format(
          CLIENT_ID, 
          CLIENT_SECRET, 
          VERSION, 
          categoryId,
          name,
          'browse',
          radius, 
          LIMIT)
            
        # make the GET request
        results = requests.get(url).json()
        lat = results['response']['geocode']['feature']['geometry']['center']['lat']
        lon = results['response']['geocode']['feature']['geometry']['center']['lng']

        results = results["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat,
            lon,
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [0]:
zurich_venues = getNearbyVenues(names=df_rest_quar['QuarLang'])


Affoltern
Albisrieden
Alt-Wiedikon
Altstetten
City
Enge
Escher Wyss
Fluntern
Friesenberg
Gewerbeschule
Hard
Hirslanden
Hirzenbach
Hochschulen
Hottingen
Höngg
Langstrasse
Leimbach
Lindenhof
Mühlebach
Oberstrass
Oerlikon
Rathaus
Saatlen
Schwamendingen-Mitte
Seebach
Seefeld
Sihlfeld
Unterstrass
Weinegg
Werd
Wipkingen
Witikon
Wollishofen


In [0]:
print(zurich_venues.shape)

zurich_venues.head()


(1155, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Affoltern,47.41814,8.5122,Restaurant Bellavista,47.410352,8.509560,Restaurant
1,Affoltern,47.41814,8.5122,Giovanni L.,47.419290,8.507070,Ice Cream Shop
2,Affoltern,47.41814,8.5122,Gasthof Löwen,47.419349,8.506212,Diner
3,Affoltern,47.41814,8.5122,Brasil - Grill im Kronenhof,47.419212,8.505541,Brazilian Restaurant
4,Affoltern,47.41814,8.5122,Pizzeria Piazza,47.419276,8.505978,Italian Restaurant


In [0]:
print('There are {} uniques categories.'.format(len(zurich_venues['Venue Category'].unique())))


There are 101 uniques categories.


In [0]:
# one hot encoding
zurich_onehot = pd.get_dummies(zurich_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
zurich_onehot['Neighborhood'] = zurich_venues['Neighborhood'] 

zurich_onehot.head()

,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,...,Taverna,Thai Restaurant,Tibetan Restaurant,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Shop,Wings Joint,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Affoltern
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Affoltern
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Affoltern
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Affoltern
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Affoltern


In [0]:
zurich_grouped = zurich_onehot.groupby('Neighborhood').mean().reset_index()
zurich_grouped

,Neighborhood,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Tapas Restaurant,Taverna,Thai Restaurant,Tibetan Restaurant,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Shop,Wings Joint
0,Affoltern,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.055556,0.000000,0.000000,0.00,0.00
1,Albisrieden,0.000000,0.000000,0.000000,0.023256,0.023256,0.000000,0.000000,0.000000,0.139535,...,0.00,0.023256,0.046512,0.00,0.023256,0.000000,0.023256,0.000000,0.00,0.00
2,Alt-Wiedikon,0.000000,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.02,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000,0.00,0.00
3,Altstetten,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.090909,...,0.00,0.000000,0.090909,0.00,0.000000,0.060606,0.000000,0.000000,0.00,0.00
4,City,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.095238,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00
5,Enge,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.02,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.00,0.00
6,Escher Wyss,0.000000,0.000000,0.000000,0.040816,0.000000,0.000000,0.000000,0.020408,0.020408,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00
7,Fluntern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00
8,Friesenberg,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.020000,0.000000,0.020000,...,0.00,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,0.020000,0.00,0.02
9,Gewerbeschule,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.060000,...,0.00,0.000000,0.020000,0.02,0.000000,0.000000,0.040000,0.020000,0.00,0.00


CLUSTERING

In [0]:
# set number of clusters
kclusters = 6

zurich_grouped_clustering = zurich_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(zurich_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([2, 2, 5, 5, 4, 5, 2, 5, 2, 5, 1, 2, 0, 5, 1, 1, 5, 3, 5, 2, 2, 5,
       5, 2, 2, 1, 5, 1, 2, 5, 2, 1, 2], dtype=int32)

In [0]:
df_rest_quar1 = zurich_venues.groupby(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude']).sum()
df_rest_quar1 = df_rest_quar1.reset_index()
df_rest_quar1

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude
0,Affoltern,47.41814,8.51220,853.517357,153.157192
1,Albisrieden,47.37398,8.49007,2037.151358,365.135506
2,Alt-Wiedikon,47.36201,8.51497,2368.163784,426.026842
3,Altstetten,47.38946,8.48533,1563.839127,280.104142
4,City,37.49445,-94.27845,787.372428,-1979.777450
5,Enge,47.36050,8.53127,2368.155962,426.460714
6,Escher Wyss,47.39052,8.51292,2322.063423,417.337185
7,Fluntern,47.38013,8.56133,1705.632218,307.990552
8,Friesenberg,47.36372,8.50417,2368.288385,425.358230
9,Gewerbeschule,47.38481,8.53011,2369.136304,426.491109


In [0]:
t = zurich_grouped[['Neighborhood']]
t.insert(0, 'Cluster Labels', kmeans.labels_)
df_zurich_clust = pd.merge(df_rest_quar1, t, how='left', on='Neighborhood')
df_zurich_clust = df_zurich_clust.fillna(0)
df_zurich_clust.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Cluster Labels
0,Affoltern,47.41814,8.51220,853.517357,153.157192,2
1,Albisrieden,47.37398,8.49007,2037.151358,365.135506,2
2,Alt-Wiedikon,47.36201,8.51497,2368.163784,426.026842,5
3,Altstetten,47.38946,8.48533,1563.839127,280.104142,5
4,City,37.49445,-94.27845,787.372428,-1979.777450,4


In [0]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_zurich_clust['Neighborhood Latitude'], df_zurich_clust['Neighborhood Longitude'], df_zurich_clust['Neighborhood'], df_zurich_clust['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters